### Load the MNIST dataset

In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml(name="mnist_784")

X, y = mnist.data, mnist.target

/home/abdallah/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


### Split it to train, validation and test sets

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=20000, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=10000, random_state=42)


### Train several classifiers

In [8]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

y_train = y_train.astype(int)
y_val = y_val.astype(int)

In [9]:
# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_val)
rf_accuracy = accuracy_score(y_val, rf_predictions)
print("Random Forest Classifier Accuracy:", rf_accuracy)


Random Forest Classifier Accuracy: 0.9677


In [10]:
# Extra-Trees Classifier
et_classifier = ExtraTreesClassifier(n_estimators=100, random_state=42)
et_classifier.fit(X_train, y_train)
et_predictions = et_classifier.predict(X_val)
et_accuracy = accuracy_score(y_val, et_predictions)
print("Extra-Trees Classifier Accuracy:", et_accuracy)

Extra-Trees Classifier Accuracy: 0.9689


### First i tried SVC but i took forever to run so i chose XGBost instead which way faster

In [11]:

# XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
xgb_classifier.fit(X_train, y_train)
xgb_predictions = xgb_classifier.predict(X_val)
xgb_accuracy = accuracy_score(y_val, xgb_predictions)
print("XGBoost Classifier Accuracy:", xgb_accuracy)

XGBoost Classifier Accuracy: 0.9348


### We create a soft voting ensemble with the pre-trained classifiers

In [12]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[
    ('random_forest', rf_classifier),
    ('extra_trees', et_classifier),
    ('xgboost', xgb_classifier)
], voting='soft')

# Train the ensemble model on the training data
ensemble.fit(X_train, y_train)

# Make predictions on the validation set
ensemble_predictions = ensemble.predict(X_val)

# Calculate accuracy on the validation set
ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
print("Ensemble (Soft Voting) Accuracy:", ensemble_accuracy)

Ensemble (Soft Voting) Accuracy: 0.9613


### Unexpectedly the accuracy has dropped down  